In [ ]:
import os
os.environ['KSU_CONF_FILE']='/databricks/conf.ini'

In [ ]:
from kensu.pyspark import init_kensu_spark
init_kensu_spark(spark, process_name='Kensu Spark Demo')

In [ ]:
# File location and type
file_location = "/FileStore/stage/AdventureWorks_Products.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# display(df)
df.write.format("delta").saveAsTable("demo.product")

In [ ]:
%sql
select * from demo.product limit 5

ProductKey,ProductSubcategoryKey,ProductSKU,ProductName,ModelName,ProductDescription,ProductColor,ProductSize,ProductStyle,ProductCost,ProductPrice
214,31,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.",Red,0,0,13.0863,34.99
215,31,HL-U509,"Sport-100 Helmet, Black",Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.",Black,0,0,12.0278,33.6442
218,23,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,Combination of natural and synthetic fibers stays dry and provides just the right cushioning.,White,M,U,3.3963,9.5
219,23,SO-B909-L,"Mountain Bike Socks, L",Mountain Bike Socks,Combination of natural and synthetic fibers stays dry and provides just the right cushioning.,White,L,U,3.3963,9.5
220,31,HL-U509-B,"Sport-100 Helmet, Blue",Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.",Blue,0,0,12.0278,33.6442


In [ ]:
# File location and type
file_location = "/FileStore/stage/AdventureWorks_Customers.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# display(df)
df.write.format("delta").saveAsTable("demo.customer")

In [ ]:
%sql
select * from demo.customer limit 5

CustomerKey,Prefix,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner
11000,MR.,JON,YANG,1966-04-08,M,M,jon24@adventure-works.com,"$90,000",2,Bachelors,Professional,Y
11001,MR.,EUGENE,HUANG,1965-05-14,S,M,eugene10@adventure-works.com,"$60,000",3,Bachelors,Professional,N
11002,MR.,RUBEN,TORRES,1965-08-12,M,M,ruben35@adventure-works.com,"$60,000",3,Bachelors,Professional,Y
11003,MS.,CHRISTY,ZHU,1968-02-15,S,F,christy12@adventure-works.com,"$70,000",0,Bachelors,Professional,N
11004,MRS.,ELIZABETH,JOHNSON,1968-08-08,S,F,elizabeth5@adventure-works.com,"$80,000",5,Bachelors,Professional,Y


In [ ]:
# File location and type
file_location = "/FileStore/stage/AdventureWorks_Sales_2017.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# display(df)
df.write.format("delta").saveAsTable("demo.order")

In [ ]:
%sql
select * from demo.order limit 5

OrderDate,StockDate,OrderNumber,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity
2017-01-01,2003-12-13,SO61285,529,23791,1,2,2
2017-01-01,2003-09-24,SO61285,214,23791,1,3,1
2017-01-01,2003-09-04,SO61285,540,23791,1,1,1
2017-01-01,2003-09-28,SO61301,529,16747,1,2,2
2017-01-01,2003-10-21,SO61301,377,16747,1,1,1


In [ ]:
%sql
select left(OrderDate, 7) as order_month,* from demo.order limit 5

order_month,OrderDate,StockDate,OrderNumber,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity
2017-01,2017-01-01,2003-12-13,SO61285,529,23791,1,2,2
2017-01,2017-01-01,2003-09-24,SO61285,214,23791,1,3,1
2017-01,2017-01-01,2003-09-04,SO61285,540,23791,1,1,1
2017-01,2017-01-01,2003-09-28,SO61301,529,16747,1,2,2
2017-01,2017-01-01,2003-10-21,SO61301,377,16747,1,1,1


In [ ]:
df = spark.sql("select b.ProductKey, b.ProductName, sum(a.OrderQuantity) AS Total_Product_Sold from demo.order a left join demo.product b on a.ProductKey = b.ProductKey group by b.ProductKey, b.ProductName order by 3 desc")
df.write.format("delta").saveAsTable("demo.product_order_count")

In [ ]:
%sql
select * from demo.product_order_count limit 5

ProductKey,ProductName,Total_Product_Sold
477,Water Bottle - 30 oz.,4565
480,Patch Kit/8 Patches,3255
528,Mountain Tire Tube,3121
223,AWC Logo Cap,2394
529,Road Tire Tube,2379


In [ ]:
df = spark.sql("select left(a.OrderDate, 7) as order_month, sum(a.OrderQuantity) AS Total_Product_Sold from demo.order a group by left(a.OrderDate, 7) order by 2 desc")
df.write.format("delta").saveAsTable("demo.order_month_agg")

In [ ]:
%sql
select * from demo.order_month_agg

order_month,Total_Product_Sold
2017-06,8260
2017-05,8199
2017-04,7680
2017-03,7327
2017-01,7020
2017-02,6828
